In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import tweepy
import time
import seaborn as sns
import quandl
quandl.ApiConfig.api_key = '_WX8xkWPcwEZ-mwbe--q'

# Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
consumer_key = "Ed4RNulN1lp7AbOooHa9STCoU"
consumer_secret = "P7cUJlmJZq0VaCY0Jg7COliwQqzK0qYEyUF9Y0idx4ujb3ZlW5"
access_token = "839621358724198402-dzdOsx2WWHrSuBwyNUiqSEnTivHozAZ"
access_token_secret = "dCZ80uNRbFDjxdU2EckmNiSckdoATach6Q8zb7YYYE5ER"

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


In [2]:
#Get ticker list from Quandl
ticker_list = pd.read_csv(r"C:\Users\robin\Desktop\Project-One\ticker_list.csv")
#Print Sample
ticker_list.head()

,ticker,description
0,WIKI/AAPL,"Apple Inc (AAPL) Prices, Dividends, Splits and..."
1,WIKI/ABC,"AmerisourceBergen Corp. (ABC) Prices, Dividend..."
2,WIKI/AA,"Alcoa Inc. (AA) Prices, Dividends, Splits and ..."
3,WIKI/ABBV,"AbbVie Inc. (ABBV) Prices, Dividends, Splits a..."
4,WIKI/ACE,"ACE Ltd (ACE) Prices, Dividends, Splits and Tr..."


In [3]:
#Slice ticker column to get ticker symbols
ticker_list['ticker_symbol'] = ticker_list['ticker'].str[5:]
#Print sample to validate data
ticker_list.head()

,ticker,description,ticker_symbol
0,WIKI/AAPL,"Apple Inc (AAPL) Prices, Dividends, Splits and...",AAPL
1,WIKI/ABC,"AmerisourceBergen Corp. (ABC) Prices, Dividend...",ABC
2,WIKI/AA,"Alcoa Inc. (AA) Prices, Dividends, Splits and ...",AA
3,WIKI/ABBV,"AbbVie Inc. (ABBV) Prices, Dividends, Splits a...",ABBV
4,WIKI/ACE,"ACE Ltd (ACE) Prices, Dividends, Splits and Tr...",ACE


In [4]:
#Get Quandl data from WIKI PRICES dataset
quandl_data = quandl.get_table('WIKI/PRICES', qopts = {'columns': ['ticker', 'date','open','close']}, 
                                ticker = 'AMZN', date = {'gte': '2018-03-15', 'lte': '2018-04-04'}, paginate=True)

In [5]:
quandl_data.head(10)

,ticker,date,open,close
None,,,,
0,AMZN,2018-03-15,1595.00,1582.32
1,AMZN,2018-03-16,1583.45,1571.68
2,AMZN,2018-03-19,1554.53,1544.93
3,AMZN,2018-03-20,1550.34,1586.51
4,AMZN,2018-03-21,1586.45,1581.86
5,AMZN,2018-03-22,1565.47,1544.10
6,AMZN,2018-03-23,1539.01,1495.56
7,AMZN,2018-03-26,1530.00,1555.86
8,AMZN,2018-03-27,1572.40,1497.05


In [6]:
# Target Account
target_user = "@BankofAmerica"

# Counter
counter = 1

# Variables for holding sentiments
sentiments = []

# Loop through 5 pages of tweets (total 100 tweets)
for x in range(50):

    # Get all tweets from home feed
    public_tweets = api.user_timeline(target_user)

    # Loop through all tweets 
    for tweet in public_tweets:

        # Print Tweets
        # print("Tweet %s: %s" % (counter, tweet["text"]))
        
        # Run Vader Analysis on each tweet
        compound = analyzer.polarity_scores(tweet["text"])["compound"]
        pos = analyzer.polarity_scores(tweet["text"])["pos"]
        neu = analyzer.polarity_scores(tweet["text"])["neu"]
        neg = analyzer.polarity_scores(tweet["text"])["neg"]
        tweets_ago = counter
        
        # Add sentiments for each tweet into an array
        sentiments.append({"Date": tweet["created_at"], 
                           "Compound": compound,
                           "Positive": pos,
                           "Negative": neu,
                           "Neutral": neg,
                           "Tweets Ago": counter})
        
        # Add to counter 
        counter = counter + 1


In [7]:
# Convert sentiments to DataFrame
sentiments_pd = pd.DataFrame.from_dict(sentiments)
sentiments_pd.head()

,Compound,Date,Negative,Neutral,Positive,Tweets Ago
0,0.6369,Thu Apr 05 16:00:32 +0000 2018,0.826,0.0,0.174,1
1,0.0000,Wed Apr 04 17:39:04 +0000 2018,1.000,0.0,0.000,2
2,0.0000,Tue Apr 03 03:22:11 +0000 2018,1.000,0.0,0.000,3
3,0.0000,Fri Mar 30 21:30:17 +0000 2018,1.000,0.0,0.000,4
4,0.5574,Thu Mar 29 18:06:14 +0000 2018,0.813,0.0,0.187,5


In [8]:
import time
for x in range(len(sentiments_pd)):
   ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(sentiments_pd["Date"].iloc[x],'%a %b %d %H:%M:%S +0000 %Y'))
   sep = ' '
   rest = ts.split(sep, 1)[0]
   
   sentiments_pd["Date"].iloc[x] = rest

C:\Users\robin\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
sentiments_pd.head(20)

,Compound,Date,Negative,Neutral,Positive,Tweets Ago
0,0.6369,2018-04-05,0.826,0.0,0.174,1
1,0.0000,2018-04-04,1.000,0.0,0.000,2
2,0.0000,2018-04-03,1.000,0.0,0.000,3
3,0.0000,2018-03-30,1.000,0.0,0.000,4
4,0.5574,2018-03-29,0.813,0.0,0.187,5
5,0.7430,2018-03-28,0.751,0.0,0.249,6
6,0.3400,2018-03-28,0.854,0.0,0.146,7
7,0.4404,2018-03-27,0.868,0.0,0.132,8
8,0.4003,2018-03-27,0.856,0.0,0.144,9
9,0.8591,2018-03-26,0.571,0.0,0.429,10


In [10]:
quandl_data.rename(columns = {'date': 'Date'}, inplace = True)

In [11]:
quandl_data.head()

,ticker,Date,open,close
None,,,,
0,AMZN,2018-03-15,1595.00,1582.32
1,AMZN,2018-03-16,1583.45,1571.68
2,AMZN,2018-03-19,1554.53,1544.93
3,AMZN,2018-03-20,1550.34,1586.51
4,AMZN,2018-03-21,1586.45,1581.86


In [13]:
quandl_data.dtypes

ticker            object
Date      datetime64[ns]
open             float64
close            float64
dtype: object

In [14]:
sentiments_pd.dtypes

Compound      float64
Date           object
Negative      float64
Neutral       float64
Positive      float64
Tweets Ago      int64
dtype: object

In [16]:
quandl_data['Date'] = quandl_data['Date'].astype('str')

In [19]:
sentiments_pd['Date'] = sentiments_pd['Date'].astype('str')

In [20]:
quandl_data.dtypes

ticker     object
Date       object
open      float64
close     float64
dtype: object

In [21]:
sentiments_pd.dtypes

Compound      float64
Date           object
Negative      float64
Neutral       float64
Positive      float64
Tweets Ago      int64
dtype: object

In [22]:
sentiments_pd.head()

,Compound,Date,Negative,Neutral,Positive,Tweets Ago
0,0.6369,2018-04-05,0.826,0.0,0.174,1
1,0.0000,2018-04-04,1.000,0.0,0.000,2
2,0.0000,2018-04-03,1.000,0.0,0.000,3
3,0.0000,2018-03-30,1.000,0.0,0.000,4
4,0.5574,2018-03-29,0.813,0.0,0.187,5


In [25]:
#Create dataframe with age range and count unique usernames
sentiments_df = sentiments_pd.groupby("Date").Compound.mean().reset_index()

In [28]:
sentiments_df.set_index(["Date"], inplace = True)
quandl_data.set_index(["Date"], inplace = True)

In [29]:
merge2 = sentiments_df.merge(quandl_data, how = "inner", left_index = True, right_index = True)

In [30]:
merge2.head()

,Compound,ticker,open,close
Date,,,,
2018-03-21,0.62490,AMZN,1586.45,1581.86
2018-03-22,0.37560,AMZN,1565.47,1544.10
2018-03-23,0.62490,AMZN,1539.01,1495.56
2018-03-26,0.80045,AMZN,1530.00,1555.86
2018-03-27,0.42035,AMZN,1572.40,1497.05
